In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy import optimize
%matplotlib qt
plt.style.use('seaborn')

In [16]:
df = pd.read_csv('data/covid.csv') 
df = df.drop('SNo', axis=1)
df['Infected'] = df['Confirmed'] - df['Deaths'] - df['Recovered']

In [17]:
def deriv(y, t, beta, gamma):
    S, I, R = y
    dSdt = -beta*S*I/N
    dIdt = beta*S*I/N - gamma*I
    dRdt = gamma*I
    return dSdt, dIdt, dRdt
    
def sir(x, beta, gamma):
    I0, R0 = 1, 0
    S0 = N - I0 - R0
    t = np.linspace(0, 100, 500)
    y0 = S0, I0, R0
    ret = odeint(deriv, y0, t, args=(beta, gamma))
    S, I, R = ret.T
    return np.interp(x, t, I)

In [23]:
def forecast(country_name):
    data = df[(df['Country/Region'] == country_name)]
    gb = data.groupby('ObservationDate')
    country = gb.sum()
    country = country.reset_index()
    infected = country['Infected'].values
    plt.scatter(np.arange(0, infected.size, 1), infected, label='infected train', color='crimson', s=12) 

    y_data = country['Infected'].values[:-4]
    x_data = np.arange(0, y_data.size, 1)
    plt.scatter(x_data, y_data, label='infected test', color='royalblue', s=12) 

    params, params_covariance = optimize.curve_fit(sir, x_data, y_data, p0=[1,1])
    x_data = np.linspace(0, 100, 500)
    I0, R0 = 1, 0
    S0 = N - I0 - R0
    y0 = S0, I0, R0
    ret = odeint(deriv, y0, x_data, args=(params[0], params[1]))
    S, I, R = ret.T

    plt.plot(x_data, I, label='infected fit', color='black')
    plt.ylabel('number of infected persons')
    plt.xlabel('days')
    plt.legend()
    plt.show()
    plt.savefig('images/sir_'+country_name+'.png', dpi=400)

In [24]:
N = 44.6e6
forecast('Spain')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in double_scalars
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in double_scalars
  """
